<a href="https://colab.research.google.com/github/silentfortin/ai-portfolio/blob/main/06-dl-pytorch-food/food_image_classification_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries for data handling, visualization, and deep learning.
import os
import requests
import zipfile
import io
import numpy as np
import random
import torch
import json
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import Counter
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report

In [ ]:
# Set random seed for reproducibility of results
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Choose device: GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using this device: {device}')

## Utility function(s)

In [ ]:
def count_images_per_class(folder_path):
    """
    Count the number of images in each class subfolder of the given directory.

    Parameters
    ----------
    folder_path : str
        Path to the parent directory containing one subdirectory per class.

    Returns
    -------
    dict
        A dictionary mapping class names to the number of images in each subfolder.
    """
    counts = {}
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            counts[class_name] = len(os.listdir(class_folder))
    return counts

In [ ]:
def evaluate(model, dataloader):
  """
  Evaluates a PyTorch model on a given validation dataset.

  Parameters
  ----------
  model : torch.nn.Module
      The model to evaluate.
  dataloader : torch.utils.data.DataLoader
      DataLoader yielding batches from the validation set.

  Returns
  -------
  val_loss : float
      Average validation loss over all batches.
  val_accuracy : float
      Classification accuracy on the validation set.
  """
  model.eval()
  val_loss, correct, total = 0, 0, 0
  with torch.no_grad():
    for images, labels in dataloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)

      loss = criterion(outputs, labels)
      val_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  return val_loss/len(dataloader), correct/total

In [ ]:
def evaluate_on_test(model, test_loader):
  """
  Runs model inference on the test set and collects all true and predicted labels.

  Parameters
  ----------
  model : torch.nn.Module
      Trained model for evaluation.
  test_loader : torch.utils.data.DataLoader
      DataLoader for the test dataset.

  Returns
  -------
  y_true : list of int
      Ground truth class indices.
  y_pred : list of int
      Predicted class indices by the model.
  """
  y_true, y_pred = [], []
  model.eval()
  with torch.no_grad():
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      y_true.extend(labels.cpu().numpy())
      y_pred.extend(predicted.cpu().numpy())
  return y_true, y_pred

In [ ]:
def set_optimizer(model, lr, weight_decay=1e-4, t_max=10):
    """
    Creates an AdamW optimizer and CosineAnnealingLR scheduler for model training.

    Parameters
    ----------
    model : torch.nn.Module
        The model with parameters to be optimized.
    lr : float
        Initial learning rate.
    weight_decay : float, optional
        L2 regularization coefficient. Default is 1e-4.
    t_max : int, optional
        Number of iterations for cosine annealing scheduler. Default is 10.

    Returns
    -------
    optimizer : torch.optim.Optimizer
        Configured AdamW optimizer.
    scheduler : torch.optim.lr_scheduler._LRScheduler
        CosineAnnealingLR scheduler.
    """
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.AdamW(trainable_params, lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max)
    return optimizer, scheduler

In [ ]:
def save_training_logs(train_losses, val_losses, train_accs, val_accs, filename):
    """
    Saves training and validation loss and accuracy logs to a JSON file.

    Parameters
    ----------
    train_losses : list of float
        List of training loss values for each epoch.
    val_losses : list of float
        List of validation loss values for each epoch.
    train_accs : list of float
        List of training accuracy values for each epoch.
    val_accs : list of float
        List of validation accuracy values for each epoch.
    filename : str
        Path to the JSON file where logs will be saved.

    Returns
    -------
    None
    """
    logs = {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "train_accs": train_accs,
        "val_accs": val_accs
    }
    with open(filename, "w") as f:
        json.dump(logs, f)
    print(f"Logs saved to {filename}")

In [ ]:
def load_training_logs(filename):
    """
    Loads training and validation loss and accuracy logs from a JSON file.

    Parameters
    ----------
    filename : str
        Path to the JSON file containing the logs.

    Returns
    -------
    train_losses : list of float
        List of training loss values per epoch.
    val_losses : list of float
        List of validation loss values per epoch.
    train_accs : list of float
        List of training accuracy values per epoch.
    val_accs : list of float
        List of validation accuracy values per epoch.
    """
    with open(filename, "r") as f:
        logs = json.load(f)
    print(f"Logs loaded from {filename}")
    return (logs["train_losses"], logs["val_losses"],
            logs["train_accs"], logs["val_accs"])


In [ ]:
def print_results(y_true, y_pred):
  """
  Prints classification results including accuracy, macro F1-score, and a classification report.

  Parameters
  ----------
  y_true : list or numpy.ndarray
      Ground truth class labels.
  y_pred : list or numpy.ndarray
      Predicted class labels.

  Returns
  -------
  None
  """
  # Accuracy
  accuracy = accuracy_score(y_true, y_pred)
  print(f'Test Accuracy: {accuracy*100:.2f}%')

  # F1-score macro
  f1 = f1_score(y_true, y_pred, average="macro")
  print(f"Macro F1-score: {f1:.4f}")

  # Classification report
  print("Classification Report:")
  print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
def plot_curves(train_losses, val_losses, train_accs, val_accs, title="Training Curves"):
    """
    Plots training and validation loss and accuracy curves over epochs.

    Parameters
    ----------
    train_losses : list of float
        Training loss at each epoch.
    val_losses : list of float
        Validation loss at each epoch.
    train_accs : list of float
        Training accuracy at each epoch.
    val_accs : list of float
        Validation accuracy at each epoch.
    title : str, optional
        Overall plot title.

    Returns
    -------
    None
    """
    plt.figure(figsize=(12,5))

    # Loss
    plt.subplot(1,2,1)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss over epochs")
    plt.legend()

    # Accuracy
    plt.subplot(1,2,2)
    plt.plot(train_accs, label="Train Acc")
    plt.plot(val_accs, label="Val Acc")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy over epochs")
    plt.legend()

    plt.suptitle(title)
    plt.show()

In [ ]:
# Download and unzip the dataset from the given URL
url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/dataset_food_classification.zip"

response = requests.get(url)
response.raise_for_status()

zip_file = zipfile.ZipFile(io.BytesIO(response.content))

zip_file.extractall("/content/")


In [ ]:
# Wrapper for using Albumentations with PyTorch Datasets
class AlbumentationsTransform:
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, img):
        # Convert PIL -> NumPy array
        img = np.array(img)
        return self.transform(image=img)["image"]

In [ ]:
# Define data augmentation pipeline for training images
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(blur_limit=(3,5), p=0.3),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Augmentation pipeline for validation/test (no random operations)
val_test_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
# Prepare datasets with class folders; apply appropriate transforms
train_dataset = ImageFolder(root='/content/dataset/train', transform=AlbumentationsTransform(train_transform))
val_dataset = ImageFolder(root='/content/dataset/val', transform=AlbumentationsTransform(train_transform))
test_dataset = ImageFolder(root='/content/dataset/test', transform=AlbumentationsTransform(val_test_transform))

# Create DataLoaders to efficiently batch and shuffle data for training
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Get the class names list
class_names = train_loader.dataset.classes

In [ ]:
# Print dataset statistics
print(f'Train samples: {len(train_dataset)}')
print(f'Valid samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')

train_counts = count_images_per_class('/content/dataset/train')
val_counts = count_images_per_class('/content/dataset/val')
test_counts = count_images_per_class('/content/dataset/test')

print("Train class distribution:", train_counts)
print("Validation class distribution:", val_counts)
print("Test class distribution:", test_counts)

In [ ]:
# Prepare tensors for image denormalization (revert normalized images to viewable format)
mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)

def denormalize(img):
    return img * std + mean

# Visualize examplse images from training set
examples = iter(train_loader)
images, labels = next(examples)

plt.figure(figsize=(8,3))

for i in range(4):
    plt.subplot(1,4,i+1)
    img = denormalize(images[i])
    img = img.permute(1, 2, 0).clamp(0,1)
    plt.imshow(img)
    plt.title(str(labels[i].item()))
    plt.axis('off')

plt.show()

# Transfer Learning Model

In [ ]:
# Early stopping utility to avoid overfitting (stop if no val improvement for X epochs)
class EarlyStopping:
  def __init__(self, patience=5):
      self.patience = patience
      self.counter = 0
      self.best_loss = None
      self.early_stop = False

  def __call__(self, val_loss):
    if self.best_loss is None or val_loss < self.best_loss:
      self.best_loss = val_loss
      self.counter = 0
    else:
      self.counter += 1
      if self.counter == self.patience:
        self.early_stop = True

In [ ]:
# Create ResNet50 baseline model for 14 food classes
num_classes = 14
baseline_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
# Freeze all parameters to use as feature extractor
for param in baseline_model.parameters():
  param.requires_grad = False
baseline_model.fc = nn.Linear(baseline_model.fc.in_features, num_classes)
baseline_model = baseline_model.to(device)

# Training setup

In [ ]:
# Define loss function and optimizer/scheduler for baseline model training
criterion = nn.CrossEntropyLoss()
# AdamW
optimizer = optim.AdamW(baseline_model.fc.parameters(), lr=1e-3, weight_decay=1e-4)
# CosineAnnealingLR
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Training and validate

In [ ]:
# Set number of epochs
epochs = 35

In [ ]:

early_stopping = EarlyStopping(patience=3)

# Initialize logs for loss/accuracy
baseline_train_losses, baseline_val_losses = [], []
baseline_train_accs, baseline_val_accs = [], []

# Training loop for baseline model
for epoch in range(epochs):
    baseline_model.train()
    correct_train, total_train = 0, 0
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = baseline_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Track training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_acc = correct_train / total_train
    train_loss = running_loss / len(train_loader)

    scheduler.step()
    val_loss, val_acc = evaluate(baseline_model, val_loader)

    # Logging
    baseline_train_losses.append(train_loss)
    baseline_val_losses.append(val_loss)
    baseline_train_accs.append(train_acc)
    baseline_val_accs.append(val_acc)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

In [ ]:
# Save baseline trained model parameters
torch.save(baseline_model.state_dict(), 'baseline_model.pth')
print("Baseline model saved")

In [ ]:
# Saving training logs
save_training_logs(baseline_train_losses, baseline_val_losses,
                   baseline_train_accs, baseline_val_accs,
                   "baseline_logs.json")

In [ ]:
# loading the baseline model
# baseline_model.load_state_dict(torch.load('baseline_model.pth'))

# moving to gpu
# baseline_model.to(device)

In [ ]:
# Plot training curves for baseline model
train_losses, val_losses, train_accs, val_accs = load_training_logs("baseline_logs.json")
plot_curves(train_losses, val_losses, train_accs, val_accs, title="Baseline Training Curves")

In [ ]:
# Evaluate baseline model on test set
baseline_model.load_state_dict(torch.load("baseline_model.pth"))
baseline_model.to(device)
y_true_base, y_pred_base = evaluate_on_test(baseline_model, test_loader)
print("Baseline Results")
print_results(y_true_base, y_pred_base)


In [ ]:
# Redefine model for fine tuning (reset and unfreeze)
fine_tuned_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = fine_tuned_model.fc.in_features
fine_tuned_model.fc = nn.Linear(num_ftrs, 14)  # 14 classi, stesso numero usato nel training

In [ ]:
# loading the trained one
# fine_tuned_model.load_state_dict(torch.load('baseline_model.pth'))

# moving to gpu
fine_tuned_model.to(device)

# Fine tuning

In [ ]:
early_stopping = EarlyStopping(patience=3)

# Logs for fine-tuning training
finetune_train_losses, finetune_val_losses = [], []
finetune_train_accs, finetune_val_accs = [], []

# Training loop for gradual unfreezing/fine tuning
for epoch in range(epochs):
    # Gradually unfreeze more layers and change learning rate as training progresses
    if epoch == 0:
        for name, param in fine_tuned_model.named_parameters():
            param.requires_grad = ("fc" in name)
        optimizer, scheduler = set_optimizer(fine_tuned_model, lr=1e-3)

    elif epoch == 5:
        for name, param in fine_tuned_model.named_parameters():
            param.requires_grad = ("fc" in name or "layer4" in name)
        optimizer, scheduler = set_optimizer(fine_tuned_model, lr=1e-4)

    elif epoch == 10:
        for name, param in fine_tuned_model.named_parameters():
            param.requires_grad = ("fc" in name or "layer4" in name or "layer3" in name)
        optimizer, scheduler = set_optimizer(fine_tuned_model, lr=1e-5)

    # Training phase for fine-tuned model
    fine_tuned_model.train()
    running_loss, correct_train, total_train = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = fine_tuned_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_acc = correct_train / total_train
    train_loss = running_loss / len(train_loader)

    scheduler.step()
    val_loss, val_acc = evaluate(fine_tuned_model, val_loader)

    # Logging
    finetune_train_losses.append(train_loss)
    finetune_val_losses.append(val_loss)
    finetune_train_accs.append(train_acc)
    finetune_val_accs.append(val_acc)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save best model for lowest val loss
    if early_stopping.best_loss is None or val_loss < early_stopping.best_loss:
        torch.save(fine_tuned_model.state_dict(), "best_finetuned_model.pth")
        print("Best fine-tuned model saved")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

In [ ]:
# Saving training logs for fined-tuned model
save_training_logs(finetune_train_losses, finetune_val_losses,
                   finetune_train_accs, finetune_val_accs,
                   "finetune_logs.json")

In [ ]:
# Load best fine-tuned parameters before final evaluation
fine_tuned_model.load_state_dict(torch.load("best_finetuned_model.pth"))
fine_tuned_model.to(device)

In [ ]:
# Plot curves for fine-tuned training
train_losses, val_losses, train_accs, val_accs = load_training_logs("finetune_logs.json")
plot_curves(train_losses, val_losses, train_accs, val_accs, title="Baseline Training Curves")

In [ ]:
y_true, y_pred = evaluate_on_test(fine_tuned_model, test_loader)

In [ ]:
# Print final test results after fine tuning
print_results(y_true, y_pred)

In [ ]:
# Confusion matrix for error analysis of fine-tuned model
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", cbar=False,
            xticklabels=class_names, yticklabels=class_names)

plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")

plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
# Show predictions vs. truth for six test images (for visual assessment)
examples = iter(test_loader)
images, labels = next(examples)
images, labels = images.to(device), labels.to(device)
outputs = fine_tuned_model(images)
_, predicted = torch.max(outputs, 1)

plt.figure(figsize=(10, 4))
for i in range(min(6, len(images))):
    plt.subplot(1, 6, i+1)
    img = denormalize(images[i].cpu())
    img = img.permute(1, 2, 0).clamp(0, 1)
    plt.imshow(img)
    plt.title(f"P: {class_names[predicted[i].cpu().item()]}\nT: {class_names[labels[i].cpu().item()]}")
    plt.axis("off")
plt.tight_layout()
plt.show()